In [398]:
import os
import requests
import json
import openai
import pickle as pkl
import requests
import json
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from time import sleep
from tqdm.auto import tqdm
import numpy as np
import ast
from sentence_transformers import SentenceTransformer
import tiktoken

In [399]:
with open('chat_gpt_iab.pkl', 'rb') as f:
    IAB_KG=pkl.load(f)

In [400]:
with open('chat_gpt_maf.pkl', 'rb') as f:
    MAF_KG=pkl.load(f)

In [401]:
len(IAB_KG)

161

In [402]:
len(MAF_KG)

938

In [428]:
with open('azure-configuration.json') as inputfile:
    azureconfig = json.load(inputfile)
openai.api_key = azureconfig['key'] 
openai.api_base = azureconfig['endpoint'] 
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

deployment_name= azureconfig['deployment_name']

response = openai.Embedding.create(
    input="Your text string goes here",
    engine=deployment_name
)
embeddings = response['data'][0]['embedding']
print(len(embeddings))

1536


In [404]:
IAB_KG[9]

{'Books and Literature': ['Fiction',
  'Non',
  'Biography',
  'Autobiography',
  'Memoir',
  'Poetry',
  'Drama',
  'Classic',
  'Bestseller',
  'Award',
  'Novels',
  'Short stories',
  'Essays',
  'Literary criticism',
  'Historical',
  'Science fiction',
  'Fantasy',
  'Romance',
  'Mystery',
  'Thriller',
  'Horror',
  "Children's literature",
  'Young adult',
  'Bildungsroman',
  'Satire',
  'Humor',
  'Tragedy',
  'Comedy',
  'Prose',
  'Verse',
  'Author',
  'Publisher',
  'Book club',
  'Library',
  'Reading']}

In [ ]:
# iab_labels=[]
# for i in IAB_KG:
#     iab_labels.append(list(i.keys())[0])

In [ ]:
# maf=[]
# for i in MAF_KG:
#     maf.append(list(i.keys())[0])

In [405]:
df_IAB=pd.read_csv('data/IAB_Evaluation.csv')

In [406]:
iab_labels=df_IAB['IAB_Label'].to_list()

In [407]:
df_MAF=pd.read_csv('data/MAF_Evaluation.csv')

In [408]:
maf=df_MAF['item'].to_list()

In [409]:
call=iab_labels+maf

In [410]:
maf

['closet',
 'rosehip',
 'appenzeller',
 'english_springer',
 'grocery',
 'siberian_husky',
 'man-eating_shark',
 'upright_piano',
 'miniature_poodle',
 'washing_machine',
 'mouth_organ',
 'bee_house',
 'beer_bottle',
 'great_white_shark',
 'border_collie',
 'hungarian_pointer',
 'kuvasz',
 'pickup',
 'globe_artichoke',
 'residence',
 'coach',
 'wireless',
 'acorn_squash',
 'dock',
 'hand_blower',
 'birdhouse',
 'medicine_cabinet',
 'church_building',
 'web_site',
 'grampus',
 'swimming_cap',
 'curly-coated_retriever',
 'desktop_computer',
 'sealyham',
 'grand',
 'wardrobe',
 'chain_armor',
 'space_heater',
 'loafer',
 'chime',
 'electric_fan',
 'bow_tie',
 'harp',
 'mosque',
 'aptenodytes_patagonica',
 'peke',
 'cd_player',
 'ocarina',
 'burrito',
 'wrecker',
 'quail',
 'lawn_cart',
 'english_springer_spaniel',
 'husky',
 'spaghetti_squash',
 'padlock',
 'cairn_terrier',
 'pyjama',
 'eating_place',
 'racing_car',
 'white_fox',
 'carbonara',
 'steel_arch_bridge',
 'tank_suit',
 'hen-of-

In [411]:
len(call)

1102

In [412]:
print(call[159:163])

['Non-Alcoholic Beverages', 'closet', 'rosehip', 'appenzeller']


In [413]:
tokenizer = tiktoken.get_encoding("cl100k_base")

In [414]:
#create doc query
expanded_call=[]

for items in tqdm(iab_labels):
    for dic in IAB_KG:
        key=list(dic.keys())[0]
        if items==key:
            list_items=dic[key]


    string_list=" ".join(list_items)
    string_list=string_list+' '+items
    string_list=string_list.replace("_", " ")
    length=len(tokenizer.encode(string_list))
    if length>=8192:
        print(items)
    expanded_call.append(string_list)

100%|██████████| 160/160 [00:00<00:00, 3068.00it/s]


In [415]:
expanded_call[23]

'video games consoles PC gaming online gaming multiplayer single player gaming community eSports virtual reality mobile gaming gaming industry game development game design game genres game mechanics gaming peripherals gaming culture game streaming game reviews gaming addiction game mods gaming tournaments game marketing Video Gaming'

In [416]:
for items in tqdm(maf):
    for dic in MAF_KG:
        key=list(dic.keys())[0]
        if items==key:
            list_items=dic[key]


    string_list=" ".join(list_items)
    string_list=string_list+' '+ items
    string_list=string_list.replace("_", " ")
    length=len(tokenizer.encode(string_list))
    if length>=8192:
        print(items)
    expanded_call.append(string_list)



100%|██████████| 942/942 [00:00<00:00, 2512.58it/s]


In [417]:
expanded_call[982]

'Lynx Wildcat Carnivore Endangered species Habitat loss Forests Prey Hunting Conservation Animal behavior Predator Adaptation Camouflage Eurasian lynx Canada lynx Iberian lynx Bobcat Lynx kittens Ecosystems Biodiversity theater curtain'

In [418]:
call[0:10]

['Hair Care',
 'Shaving',
 'Motorcycles',
 "Women's Formal Wear",
 "Women's Clothing",
 'Retail Industry',
 'Comedy Events',
 'Household Supplies',
 'Home Security',
 'Polar Travel']

In [419]:
total_embeddings_old=[]

In [475]:
total_embeddings=[]

In [424]:
def create_embeddings(item,total_embeddings):
    deployment_name= azureconfig['deployment_name']

    response = openai.Embedding.create(
        input=item,
        engine=deployment_name
    )
    embeddings = response['data'][0]['embedding']
    total_embeddings.append(embeddings)

    

In [471]:
with open('data/two_hop_expanded.pkl', 'rb') as f:
    expanded_call=pkl.load(f)

In [476]:
for items in tqdm(expanded_call):
    create_embeddings(items,total_embeddings)

100%|██████████| 1102/1102 [02:33<00:00,  7.19it/s]


In [477]:
len(total_embeddings)

1102

In [478]:
iab_embeddings=total_embeddings[0:160]

In [479]:
maf_embeddings=total_embeddings[160:]

In [482]:
def calculate_precision_recall(true_labels, predictions):
    true_positives = sum(1 for pred in predictions if pred in true_labels)
    false_positives = len(predictions) - true_positives
    false_negatives = len(true_labels) - true_positives
    
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    
    return precision, recall

In [483]:
def calculate_f1_score(precision, recall):
    if precision + recall == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [484]:
nodes=df_MAF['item'].to_list()
node_id=df_MAF['node_id'].to_list()
IAB_Candidates=df_MAF['IAB_Candidates'].to_list()
H_Mappings=df_MAF['H_Scores'].to_list()

In [485]:
cos_sim=cosine_similarity(np.array(iab_embeddings[90]).reshape(1,-1),np.array(maf_embeddings[12]).reshape(1,-1))
print(cos_sim)

[[0.72195021]]


In [530]:
top_n_labels_new=[]
top_n_cosines_new=[]
top_n_nodes=[]
top_n_node_ids=[]
top_n_IAB_Candidates=[]
top_n_H_scores=[]
top_n_labels=[]
top_n_cosines=[]

for i in tqdm(range(len(maf_embeddings))):
    similarities=[]
    for j in range(len(iab_embeddings)):
        cos_sim=cosine_similarity(np.array(iab_embeddings[j]).reshape(1,-1),np.array(maf_embeddings[i]).reshape(1,-1))
        similarities.append([call[160+j],call[j],cos_sim])
    
    similarities.sort(key=lambda x: x[2], reverse=True)


    #threshold logic needed
    similarities=[x for x in similarities if x[2]>=0.740]

    
    
    if len(similarities)==0:
        similarities.append(['No_Pred','No_Pred',[[0]]])
    
    temp_iab_labels=[]
    temp_cosines=[]


    for items in similarities:
        #print(items[1])
        temp_iab_labels.append(items[1])
        temp_cosines.append(items[2][0])
        
        top_n_labels_new.append(items[1])
        top_n_cosines_new.append(items[2][0])
        top_n_nodes.append(nodes[i])
        top_n_node_ids.append(node_id[i])
        top_n_IAB_Candidates.append(IAB_Candidates[i])
        top_n_H_scores.append(H_Mappings[i])


    
    top_n_labels.append(temp_iab_labels)
    top_n_cosines.append(temp_cosines)


100%|██████████| 942/942 [01:14<00:00, 12.56it/s]


In [531]:
len(top_n_labels)

942

In [532]:
len(top_n_labels_new)

90880

In [533]:
len(top_n_cosines)

942

In [534]:
MAF_predictions=df_MAF['IAB_Candidates'].to_list()

In [535]:
MAF_predictions[3]

"['Dogs']"

In [536]:
candidate_count=0
hit_count=0
precision=0
recall=0
count=0
f1=0

for predictions, labels in zip(MAF_predictions,top_n_labels):
    count+=1
    proper=ast.literal_eval(predictions)
    prec, reca=calculate_precision_recall(predictions, labels)

    f=calculate_f1_score(prec, reca)
    precision+=prec
    recall+=reca
    f1+=f


    candidate_count+=len(proper)
    for items in labels:
        if items in proper:
            hit_count+=1

In [537]:
hit_count/candidate_count

0.8633610900832702

In [538]:
precision/942

0.029679901153189588

In [539]:
recall/942

0.07043897367630077

In [540]:
f1/942

0.03269189868365176

In [ ]:
dic={'item':top_n_nodes, 'node_id': top_n_node_ids, 'IAB_Candidates': top_n_IAB_Candidates,
'predictions': top_n_labels_new, 'cosine_predicted': top_n_cosines_new, "H_Mapping_scores": top_n_H_scores }

In [ ]:
df_MAF_new=pd.DataFrame.from_dict(dic)

In [ ]:
df_MAF_new.to_excel('results/OpenAI_results_0.755_new.xlsx')

In [ ]:
df_MAF_new.shape

In [ ]:
#Top 100 in predictions but not in embedding

In [ ]:
df_MAF_new=df_MAF_new.sort_values(by='cosine_predicted', ascending=False)

In [ ]:
df_MAF_new.iloc[0]

In [ ]:
top_100_node=[]
top_100_embedding_pred=[]
top_100_H_pred=[]
top_100_cos_pred=[]
top_100_H_score=[]

In [ ]:
items=df_MAF_new['item'].to_list()
IAB_Candidates=df_MAF_new['IAB_Candidates'].to_list()
predictions=df_MAF_new['predictions'].to_list()
cosine_predicted=df_MAF_new['cosine_predicted'].to_list()
H_Mapping_scores=df_MAF_new['H_Mapping_scores'].to_list()

In [ ]:
for item,IAB,pred,cos,H_map in tqdm(zip(items,IAB_Candidates,predictions,cosine_predicted,H_Mapping_scores)):
    pred_list=ast.literal_eval(IAB)
    if pred not in pred_list:
        top_100_node.append(item)
        top_100_embedding_pred.append(pred)
        top_100_H_pred.append(IAB)
        top_100_cos_pred.append(cos)
        top_100_H_score.append(H_map)
    if len(top_100_node)==100:
        break

In [ ]:
top_100_dic={'items':top_100_node, 'Embedding Predictions': top_100_embedding_pred,
 "H_Mapping_Pred": top_100_H_pred, 'Predicted Cosine': top_100_cos_pred, 
 'H Mapping Cosine': top_100_H_score}

In [ ]:
df_top_100=pd.DataFrame.from_dict(top_100_dic)

In [ ]:
df_top_100.to_excel('results/OpenAI_Top_100_Not_In_H.xlsx', index=False)

In [ ]:
#Top 100 in H but not in Embedding

In [ ]:
print(top_n_labels[5])
print(top_n_cosines[5])

In [ ]:
nodes=df_MAF['item'].to_list()
# node_id=df_MAF['node_id'].to_list()
IAB_Candidates=df_MAF['IAB_Candidates'].to_list()
H_Mappings=df_MAF['H_Scores'].to_list()

In [ ]:
mid_node=[]
mid_IAB=[]
mid_label=[]
mid_cos=[]
mid_H_map=[]

for node, IAB, label, cos, H_map in tqdm(zip(nodes,IAB_Candidates,top_n_labels,top_n_cosines,H_Mappings)):
    IAB_list=ast.literal_eval(IAB)
    H_Map_list=ast.literal_eval(H_map)
    for IAB_item, H_Map_item in zip(IAB_list,H_Map_list):
        mid_node.append(node)
        mid_IAB.append(IAB_item)
        mid_label.append(label)
        mid_cos.append(cos)
        mid_H_map.append(H_Map_item)

In [ ]:
dic={'item':mid_node, 'IAB_Candidates': mid_IAB,
'predictions': mid_label, 'cosine_predicted': mid_cos, "H_Mapping_scores": mid_H_map}

In [ ]:
df_lol_100=pd.DataFrame.from_dict(dic)

In [ ]:
df_lol_100=df_lol_100.sort_values(by='H_Mapping_scores', ascending=False)

In [ ]:
df_lol_100.head()

In [ ]:
top_100_node=[]
top_100_embedding_pred=[]
top_100_H_pred=[]
top_100_cos_pred=[]
top_100_H_score=[]

In [ ]:
items=df_lol_100['item'].to_list()
IAB_Candidates=df_lol_100['IAB_Candidates'].to_list()
predictions=df_lol_100['predictions'].to_list()
cosine_predicted=df_lol_100['cosine_predicted'].to_list()
H_Mapping_scores=df_lol_100['H_Mapping_scores'].to_list()

In [ ]:
for item,IAB,pred,cos,H_map in tqdm(zip(items,IAB_Candidates,predictions,cosine_predicted,H_Mapping_scores)):
    #pred_list=ast.literal_eval(pred)
    if IAB not in pred:
        top_100_node.append(item)
        top_100_embedding_pred.append(pred)
        top_100_H_pred.append(IAB)
        top_100_cos_pred.append(cos)
        top_100_H_score.append(H_map)
    if len(top_100_node)==100:
        break

In [ ]:
top_100_dic={'items':top_100_node, 'Embedding Predictions': top_100_embedding_pred,
 "H_Mapping_Pred": top_100_H_pred, 'Predicted Cosine': top_100_cos_pred, 
 'H Mapping Cosine': top_100_H_score}

In [ ]:
df_top_100=pd.DataFrame.from_dict(top_100_dic)

In [ ]:
df_top_100.to_excel('results/Top_100_Not_In_OpenAI.xlsx', index=False)